# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
months_num = {
    'Jan-2019':1, 
    'Feb-2019':2,
    'Mar-2019':3, 
    'Apr-2019':4,
    'May-2019':5
}

In [6]:
pd.set_option('display.max_columns', None)
df.head()
df['next_pymnt_d'].value_counts()

May-2019    42449
Apr-2019    26368
Name: next_pymnt_d, dtype: int64

In [7]:
loan_data = df.copy()
loan_data = pd.get_dummies(loan_data , columns = ['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])
loan_data['loan_status'] = loan_data['loan_status'].apply(lambda x: 1 if x == 'high_risk' else 0)
loan_data['issue_d'] = loan_data['issue_d'].apply(lambda x: months_num[x])
loan_data['next_pymnt_d'] = loan_data['next_pymnt_d'].apply(lambda x: months_num[x])
loan_data_target = loan_data['loan_status']
loan_data = loan_data.drop('loan_status', axis = 1)
loan_data.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,3,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,10204.87,10204.87,355.29,355.29,295.13,60.16,0.0,0.0,0.0,375.35,5,0.0,1.0,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,0,1,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,3,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,24293.13,24293.13,873.53,873.53,706.87,166.66,0.0,0.0,0.0,929.09,5,0.0,1.0,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,1,0,0,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,3,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,19636.79,19636.79,485.44,485.44,363.21,122.23,0.0,0.0,0.0,529.88,5,0.0,1.0,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,1,0,0,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,3,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,9719.34,9719.34,335.33,335.33,280.66,54.67,0.0,0.0,0.0,353.55,5,0.0,1.0,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,0,1,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,3,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,21614.73,21614.73,484.36,484.36,385.27,99.09,0.0,0.0,0.0,520.39,5,0.0,1.0,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,1,0,0,1,0,0,1,0,1,1,0,1,1


## Split the Data into Training and Testing

In [8]:
# Create our features
X = loan_data

# Create our target
y = loan_data_target

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.00000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,24.666347,16018.845833,16015.919519,970.639473,970.431151,658.748750,311.855687,0.035072,0.0,0.0,517.202218,4.616839,0.016188,1.0,0.0,188.664298,1.634976e+05,1.023047,3.290481,0.843730,1.909644,14.303152,4.324044e+04,69.21191,1.240042,2.622245,6297.491710,55.702472,42431.895360,1.382071,1.687795,2.189023,4.827572,14142.936193,16991.852667,48.330667,0.006830,2.347065,125.468605,175.136841,14.294215,7.179781,1.441882,24.714954,7.688028,0.472485,3.806123,5.648038,5.135156,7.414403,9.500153,8.630818,13.517663,5.653036,12.571995,0.0,0.0,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.009285,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,1.0,0.123879,0.876121,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,12.318917,10002.695495,10004.985940,1229.563301,1229.569864,1126.237377,279.786954,1.086720,0.0,0.0,816.021053,0.486161,0.134774,0.0,0.0,1808.457195,1.723699e+05,1.176782,2.965579,0.970543,1.544994,13.972619,4.956130e+04,22.33711,1.471745,2.502608,5702.281757,18.495480,37166.229316,1.543220,2.852664,2.396186,3.145377,15863.878993,20447.276820,28.106287,0.087327,301.312407,56.968853,99.149467,17.417118,6.303066

In [10]:
# Check the balance of our target values
# YOUR CODE HERE
Counter(y)

Counter({0: 68470, 1: 347})

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , random_state = 1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
#Resampling 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# X_resampled , y_resampled = ros.fit_resample(X_train , y_train)
X_resampled , y_resampled = SMOTE(random_state=1 , sampling_strategy =1.0).fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 51366, 1: 51366})

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE

from sklearn.ensemble import GradientBoostingClassifier
forest_model = GradientBoostingClassifier(n_estimators = 100,
                                          learning_rate = 0.75,
                                          max_depth = 1,
                                          random_state=1)
    
ml = forest_model.fit(X_resampled, y_resampled)
predictions = ml.predict(X_test_scaled) 

In [20]:
print(X_test_scaled.shape)

(17205, 92)


In [21]:
print(forest_model.score(X_resampled, y_resampled))

0.9950940310711366


In [22]:
print(forest_model.score(X_test_scaled, y_test))

0.9945945945945946


In [23]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test , predictions)

0.7216906588928304

In [24]:
results = pd.DataFrame({'Actual': y_test , 'Predictions': predictions})
results.head()

,Actual,Predictions
7034,0,0
38804,0,0
65533,0,0
37736,0,0
6390,0,0


In [25]:
# Display the confusion matrix
# YOUR CODE HERE
cm = confusion_matrix(y_test , predictions)
cm_df = pd.DataFrame(cm , index = ['Actual 0', 'Actual 1'], columns = ['Predicted 0', 'Predicted 1'])
cm_df

,Predicted 0,Predicted 1
Actual 0,17067,37
Actual 1,56,45


In [26]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test , predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.45      1.00      0.67      0.47     17104
          1       0.55      0.45      1.00      0.49      0.67      0.42       101

avg / total       0.99      0.99      0.45      0.99      0.67      0.47     17205



In [27]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = forest_model.feature_importances_
sorted(zip(forest_model.feature_importances_, X.columns), reverse = True)

[(0.2599541373933393, 'total_rec_prncp'),
 (0.19445945378959106, 'issue_d'),
 (0.11965057847808543, 'next_pymnt_d'),
 (0.06176286443048933, 'total_rec_int'),
 (0.054165541380899566, 'total_cu_tl'),
 (0.04245280291964156, 'loan_amnt'),
 (0.04050851422909287, 'int_rate'),
 (0.029154480245093723, 'inq_last_12m'),
 (0.027210674615373777, 'verification_status_Not Verified'),
 (0.02271729784436877, 'verification_status_Source Verified'),
 (0.01903105598794767, 'pct_tl_nvr_dlq'),
 (0.018388719569619027, 'num_accts_ever_120_pd'),
 (0.013212898804187898, 'verification_status_Verified'),
 (0.010426298008999775, 'inq_last_6mths'),
 (0.010025650601371252, 'open_acc_6m'),
 (0.009498563067184008, 'open_rv_24m'),
 (0.009349337700061423, 'open_rv_12m'),
 (0.0077310994310275565, 'total_pymnt'),
 (0.006683692404103444, 'last_pymnt_amnt'),
 (0.0054535433834255655, 'inq_fi'),
 (0.00536796521243395, 'percent_bc_gt_75'),
 (0.005113518025350572, 'installment'),
 (0.005011323849934674, 'mo_sin_rcnt_rev_tl_op'

### Easy Ensemble Classifier

In [33]:
# Train the Classifier
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
forest_model_2 = RandomForestClassifier(n_estimators=100 , random_state =1)
ml_2 = forest_model_2.fit(X_resampled , y_resampled)
predictions_2 = ml_2.predict(X_test_scaled )

In [34]:
print(forest_model_2.score(X_resampled, y_resampled))

1.0


In [35]:
print(forest_model_2.score(X_test_scaled , y_test))

0.9955245568148794


In [36]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
balanced_accuracy_score(y_test , predictions_2)

0.653260716038863

In [31]:
# Display the confusion matrix
# YOUR CODE HERE
cm_2 = confusion_matrix(y_test , predictions_2)
cm_2_df = pd.DataFrame({'Actual': y_test, 'Predictions': predictions_2})
cm_2_df.head()

,Actual,Predictions
7034,0,0
38804,0,0
65533,0,0
37736,0,0
6390,0,0


In [32]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test , predictions_2))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      0.31      1.00      0.55      0.33     17104
          1       0.84      0.31      1.00      0.45      0.55      0.29       101

avg / total       0.99      1.00      0.31      0.99      0.55      0.33     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
#1 Model 1
#2 Model 1 
#3 Model 1 
#4 (0.07030706590014925, 'total_rec_prncp'),
 #(0.06658042622340525, 'total_rec_int'),
 #(0.06607202880445075, 'last_pymnt_amnt')